In [1]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pylab as plt
import sys, os, glob
import numpy as np
import subprocess

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

from IPython.display import HTML
import xml.etree.ElementTree as ET
try:
    tree = ET.parse(os.environ['HADOOP_CONF_DIR'] + '/yarn-site.xml')
except IOError:
    raise IOError("Can't find the yarn configuration -- is HADOOP_CONF_DIR set?")
root = tree.getroot()
yarn_web_app = root.findall("./property[name='yarn.resourcemanager.webapp.address']")[0].find('value').text
yarn_web_app_string = "If this works successfully, you can check the <a target='_blank' href='http://{yarn_web_app}'>YARN application scheduler</a> and you should see your app listed there. Clicking on the 'Application Master' link will bring up the familiar Spark Web UI. "

plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

## Set up and launch the Spark runtime

Remember from the previous notebook that we have a saved configuration in `./spark_config/` -- so all we need to do is set the `SPARK_CONF_DIR` environment variable and our default configuration will be used: 

In [2]:
# specify the configuration directory
os.environ['SPARK_CONF_DIR'] = os.path.realpath('./spark_config')

# how many cores do we have for the driver
ncores = int(os.environ.get('LSB_DJOB_NUMPROC', 1)) 


# here we set the memory we want spark to use for the driver JVM
os.environ['SPARK_DRIVER_MEMORY'] = '%dG'%(ncores*0.7)

# we have to tell spark which python executable we are using
os.environ['PYSPARK_PYTHON'] = subprocess.check_output('which python', shell=True).rstrip()

import findspark
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext

conf = SparkConf()

conf.set('spark.driver.maxResultSize', '4g')
conf.set('spark.executor.instances', 10)
conf.set('spark.yarn.am.cores', ncores)
conf.set('spark.metrics.conf', '/cluster/home03/sdid/roskarr/spark_graphite_conf/metrics.properties')
conf.set('spark.storage.memoryFraction', '0.3')
conf.set('spark.storage.shuffleFraction', '0.2')

sc = SparkContext(master='yarn-client', conf=conf)

# Uncovering World Events using Twitter Hashtags
## ... and learning about Spark `DataFrames` along the way

In this notebook, we will use temporal information about Twitter hashtags to uncover world events. In case someone isn't familiar with [Twitter](http://twitter.com): a "tweet" is a 140 character message broadcast into the world.  A hashtag is a part of a tweet, a single words preceded by a "#", and is used to indicate the topic of the tweet. If you want to find out about photography, for example, you might search for the [#photography](https://twitter.com/search?q=%23photography&src=typd) hashtag. 

The idea here is that when an event is happening and people are having a conversation about it on Twitter, a set of uniform hashtags that represent the event spontaneously evolves and Twitter users then use those hashtags to communicate with one another. Some hashtags, like "#RT" for "retweet" or just "#retweet" are used frequently and don't tell us much about what is going on. But a sudden appearance of a hashtag like "#oscars" probably indicates that the oscars are underway. For a particularly cool example of this type of analysis, check out this [blog post about earthquake detection using Twitter data](https://blog.twitter.com/2015/usgs-twitter-data-earthquake-detection) (although they search the text and not necessarily hashtags).

We will do some simple statistics to try and find some of these event hashtags. This entire notebook uses the Spark `DataFrame`, which is a higher-level abstraction built on top of the `RDD` that we have grown to love. `DataFrame`s are designed to be efficient at handling tabular, structured data. Anything that has columns and rows, essentially. 

As we'll see below, the `DataFrame` allows us to process data efficiently and often use fewer lines of code. By using the optimized built-in functions, we can benefit from the performance saving of the pure scala implementation while still using Python. When something more complex is required that is not easily expressed in terms of `DataFrame` operations, we can temporarily switch into `RDD` mode. For some more background, see the [DataFrame API blog post](https://databricks.com/blog/2015/02/17/introducing-dataframes-in-spark-for-large-scale-data-science.html). 

<div class="alert alert-info">
<p><strong>Viewer discretion is advised!</strong></p> 

<p>The internet is, in general, not a safe place. The data used in this tutorial is an unfiltered stream of what people all over the world tweet. The chances of there being something more or less offensive in this sea of tweets are pretty high, so be prepared to perhaps see some foul language and/or sexual references. </p>
</div>

## Initialization

Using `DataFrame`s, our entry point into the Spark universe is the `SQLContext` or the equivalent `HiveContext`:

In [3]:
from pyspark.sql import SQLContext, HiveContext

hc = HiveContext(sc)

We'll load the data off the disk, but only for the last three months of the year. 

In [4]:
%%time
data = hc.read.parquet('/user/roskarr/twitter/2014_10*')

CPU times: user 7 ms, sys: 5 ms, total: 12 ms
Wall time: 24.6 s


In [ ]:
data.count()

`data` is now a `DataFrame` object, which is essentially a collection of `Row` objects. Each `Row` object contains data for whatever columns are defined in the `DataFrame`. Here is a critical difference between `DataFrames` and `RDD`s: each column has an associated data type. While in dealing with an `RDD` we relied on Python to convert types, here each column has a specified data type. This means that:

a) we have to be a bit more careful about what we are doing and 

b) the execution engine can optimize our calculations because the data is no longer a black box. 

### Importing libraries and types for `DataFrame` API

The next few cells are a bit lengthy and complicated so you can just treat them as black boxes for now. First, we just import some necessary libraries and classes, then a series of functions are defined and executed on the `data` to give us a `DataFrame` of hashtags. 

In [6]:
import pyspark.sql.functions as func
from pyspark.sql import Row, Window
from pyspark.sql.types import IntegerType, ArrayType, StringType, NullType, LongType, StructField, StructType, DateType, DataType, DateConverter, DatetimeConverter, TimestampType, BooleanType
import datetime

### Custom functions to extract hashtags

The twitter data is rather complex and includes many different kinds of fields that we will not be using here. We will also change the format of the time string to something that Spark will have an easier time understanding. 

Here we define a few "user defined functions" (UDFs) -- these are crucial for working with `DataFrame`s and we will discuss them in more detail in a bit. 

In [7]:
# change the date format to YYYY-MM-DD HH:MM:SS
convert_date_string = func.udf(lambda date_string: datetime.date.strftime(datetime.datetime.strptime(date_string, '%a %b %d %H:%M:%S +0000 %Y'),'%Y-%m-%d %H:%M:%S'), StringType())

# make UDF for converting the date string to a datetime object
datetime_udf = func.udf(lambda date_string: datetime.strptime(date_string, '%a %b %d %H:%M:%S +0000 %Y'), DateType())

# make a UDF 
hash_text_udf = func.udf(lambda row: [r.text.lower() for r in row], returnType=ArrayType(StringType()))

Npartitions = sc.defaultParallelism*5

# only keep the tweets with at least one hashtag
hashtag_df = (data.select('created_at', 'entities.hashtags')
                .filter(func.size('hashtags') > 0)
                .withColumn('hash_text', hash_text_udf('hashtags'))
                .select(convert_date_string('created_at').alias('date'), func.explode('hash_text').alias('hashtag'))
                .withColumn('hashtag', func.lower(func.col('hashtag')))
                .repartition(Npartitions))
#hashtag_df.cache()

#### A look at the raw data

We now have a `DataFrame` consisting of a timestamp and hashtag. The `show()` method is extremelly useful here -- it functions like `take()` for an RDD, but it formats the output nicely so it's more easily readable. 

<div class="alert alert-info">
<p>We're dealing with ~3 Tb of raw data here, so some of these operations might take a little time...</p>
</div>

In [ ]:
hashtag_df.show(truncate=False)

Underneath the `DataFrame` object is an RDD of `Row` objects, which are just containers used by `DataFrame`:

In [ ]:
hashtag_df.take(5)

The structure of `Row`s is revealed by the `DataFrame` schema: 

In [ ]:
hashtag_df.printSchema()

In `DataFrame`s, you can think of the `select` method to be analogous to `map` in an `RDD`. In its simplest form, `select` is just used to, well, trim down the `DataFrame` to the columns that are needed: 

In [ ]:
hashtag_df.select('date').show(5)

You can select multiple columns: 

In [ ]:
hashtag_df.select('date', 'hashtag').show(5)

In addition, we can pass functions to `select` that will operate on existing columns and define a new column. For example, lets use the `month` function to extract the month from the date string: 

In [ ]:
hashtag_df.select('hashtag', func.month('date')).show(5)

If some extra functionality is needed, we can define a custom "user defined function" (UDF) that uses the standard python interpreter. The function has to be registered as a UDF using [`pyspark.sql.functions.udf`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.udf). For example: 

In [ ]:
def string_length(x):
    return len(x)

string_length_udf = func.udf(string_length, LongType())

In [ ]:
hashtag_df.select('hashtag', string_length_udf('hashtag')).show(5)

### `DataFrame` superpower:  `groupBy`

`DataFrame`'s `groupBy()` method to allows you to collect together rows that share a common value. This is a powerful method that allows you to perform aggregations on your data quickly. It can serve as a replacement to `reduceByKey` that is used on `RDD`s and requires considerably less wrangling with keys and values. 

This is also where things get a little confusing: unlike other `DataFrame` and `RDD` methods, `groupBy` doesn't return another `DataFrame`. Instead, it returns a `GroupedData` object, which has very limited functionality. It can *only* perform the dedicated aggregation functions `avg`, `count`, `max`, `mean`, `min`, and `sum`. You can specify a column (or columns) for the aggregation for all of these except `count`, which just counts the number of rows in each group. These methods all return another `DataFrame`, so the `GroupedData` is a sort of intermediate stage. 

Here, we will try to find out how many hashtags are tweeted each minute, so we will first create a custom UDF that calculates minute-of-the-year for each hashtag. Then we will use `groupBy` aggregation methods to obtain counts. 

In [8]:
minute_of_the_year = (func.dayofyear('date')*60*24 + func.hour('date')*60 + func.minute('date')).alias('moy')

In [9]:
moy_df = hashtag_df.select('hashtag', minute_of_the_year)

In [10]:
moy_df.show(5)

+-------------------+------+
|            hashtag|   moy|
+-------------------+------+
|             産めないのか|395029|
|                m5s|395029|
|         غرد_بالخير|395029|
|تغريدة_تتمنى_تكتبها|395220|
|              boxeo|395220|
+-------------------+------+
only showing top 5 rows



In [11]:
moy_group = moy_df.groupBy('moy')
moy_group

Note that `moy_group` is a `GroupedData` instance - it contains groups of rows that all have the same value of `'moy'`. Calling `count()` will give us back a `DataFrame`:

In [12]:
moy_count = moy_group.count()
moy_count

DataFrame[moy: int, count: bigint]

In [13]:
moy_count.show()

+------+-----+
|   moy|count|
+------+-----+
|437831|  946|
|438031|  744|
|436431| 1163|
|424631| 1060|
|436031| 1175|
|401831|  952|
|431631| 1413|
|435031| 1163|
|436631|  751|
|399231|  777|
|433231| 1180|
|409231|  874|
|398031|  888|
|422831| 1657|
|395631| 1102|
|402031|  885|
|426631|  865|
|432431|  801|
|412631| 1167|
|406631|  730|
+------+-----+
only showing top 20 rows



We can collect this "reduced" data into a Pandas dataframe and plot it: 

In [14]:
moy_data = moy_count.toPandas()

In [ ]:
fig, ax = plt.subplots()
plt.plot(moy_data['moy'][:1000], moy_data['count'][:1000], 'o', alpha=.5)
plt.xlabel('minute of year'); plt.ylabel('hashtags per minute')

For fun, lets do a quick aside and check for any obvious periods in there...

In [ ]:
import scipy.fftpack

N = len(moy_data['count'])
T = (moy_data['moy'].max() - moy_data['moy'].min())/N

yf = scipy.fftpack.fft(moy_data['count'])

xf = np.linspace(0.0, 1.0/(2.0*T), N/2)

fig, ax = plt.subplots()
ax.plot(1./xf/60.,2.0/N * np.abs(yf[0:N/2]))
ax.set_xlim(0,36)
ax.set_xlabel('period (hours)'); ax.set_ylabel('power');

A lot of structure in there! some of it might be harmonics, but looking into periodicity on Twitter [has been done before](http://theory.stanford.edu/~nmishra/Papers/groupChatsOnTwitter.pdf) (obviously).

### Simple word count revisited

OK, back to Spark. Now that we have a `DataFrame` consisting of `(timestamp, hashtag)` columns, lets do a simple word count (or a hashtag count in this case) using the `DataFrame` API. To accomplish this, we'll once again use the `groupBy` method of the `DataFrame`:

In [ ]:
%%time 
hashtag_df.groupBy('hashtag').count().sort('count', ascending=False).show(25, False)

In [ ]:
%%time
res = (hashtag_df.map(lambda r: (r.hashtag, 1))
           .reduceByKey(lambda a,b:a+b)
           .sortBy(lambda (k,count): count, ascending=False)).take(25)
for x in res: 
    print(x[0], x[1])

 As we can see here, there are lots os languages represented on Twitter! This is very interesting by itself, but for now lets focus on just plain ascii hash tags which we are more likely to understand: 

In [ ]:
# define a helper UDF that can be used to filter non-ascii hashtags
def is_ascii(s):
    if s is not None: 
        return all(ord(c) > 0 and ord(c) < 128 for c in s)
    else : 
        return False
is_ascii_udf = func.udf(is_ascii, BooleanType())

In [ ]:
hashtag_df = hashtag_df.filter(is_ascii_udf('hashtag'))

### Daily hashtag trends

Now lets do something slightly more complicated and incorporate the time data into our analysis. 

To get information about daily hashtag usage, we must first convert the date string into a number representing day of the year. The [Spark DataFrame functions module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) provides a function [dayofyear](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.dayofyear) that will do this for us and we can use it just like we used the User Defined Functions (UDFs) above. 

In [ ]:
daily_hashtag = hashtag_df.select(func.month('date').alias('month'),
                                  func.weekofyear('date').alias('week'),
                                  func.dayofyear('date').alias('day'), 
                                  'hashtag')
daily_hashtag.show(5)

We can now use the same technique as above using `groupBy` to get daily counts for each hashtag. Since we want to satisfy two conditions (counts per day and counts per hashtag), we simply give `groupBy` two columns:

In [ ]:
day_counts = daily_hashtag.groupby('day', 'hashtag', 'week').count()

In [ ]:
%time day_counts.sort('day').count()

#### Important aside concerning `count()` (read this while waiting for the above line to complete)

The use of `count()` here is very confusing -- in the first of the two cells above, `count()` is a method of a [GroupedData](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) object, which performs a counting of items in each group. Our grouping here is defined by week, day, and hashtag, so calling `count()` gives us the number of times a hashtag appears each day of the year (week here has no effect since days are unique anyway). To make it even more confusing, `count()` in this case actually returns a `DataFrame`, with "count" as one of the columns. 

However, we are much more used to seeing `count()` as an `RDD` method! This is done in the second of the two cells above. There, it returns a single number representing the number of elements in the `RDD` (or `DataFrame`), which is much different. 

#### `RDD` analogy:

The same counting of daily occurences of each hashtag can be done with an `RDD` operation using a `map` and `reduceByKey`, though it might look a bit messier: 

In [ ]:
# TODO
day_counts_rdd = daily_hashtag.rdd.map(lambda r: ((r.day, r.hashtag), 1)).reduceByKey(lambda a,b: a+b)

In [ ]:
%time day_counts_rdd.sortBy(lambda ((day, hashtag), count): day).count()

Wow, that takes way longer! The `DataFrame` API avoids shipping data back and forth to the Python intepreter (and can do some extra optimizations) so the performance difference can be significant, as we can see above. 

Lets make sure that the results are the same by looking up a hashtag in both places:

In [ ]:
day_counts_rdd.lookup((274,'retweet'))

In [ ]:
day_counts.filter((day_counts.hashtag == 'retweet') & (day_counts.day == 274)).show()

At this point we should persist `day_counts` in memory since we'll be using it later on. 

In [ ]:
day_counts.cache()

We can get a quick idea of the most popular hashtags by looking at the weekly averages of daily hashtag counts:

In [ ]:
day_counts.groupBy('week', 'hashtag')\
           .avg('count')\
           .sort('avg(count)', ascending=False)\
           .show(20)

Apparently, the [MTV Europe Music Awards](https://en.wikipedia.org/wiki/2014_MTV_Europe_Music_Awards) happened in late 2014... (and [yet another boy band](https://en.wikipedia.org/wiki/One_Direction) is on the scene?)

## Using `window` functions

### top daily hashtags

Now lets say we want to know the top tweets for each day. This is a non-trivial thing to try and compute using a standard RDD, but the `DataFrame` API gives us ["window"](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) functions that let us do it relatively easily. 

A window function defines how the data should be grouped and ordered. Then, when using either a ranking or analytic function (see the list in the [window functions article](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html)) we can specify which window to run the function over. 

In this example, we will define a window function, `daily_window`, over the `'day'` column so our data will be grouped by day. Applying `daily_window` to the built-in `rank` function will give us `daily_rank`, a function that will yield a new column `'rank'` that will hold the rank of each hashtag on a given day based on its counts (in descending order).

In the end, this is a fairly complicated operation achieved in just a few lines of code! 

First, specify the quantity to window over -- in this case we want rankings per day:

In [ ]:
daily_window = Window.partitionBy('day')

Second, define a function that we want to compute over the window

Here, we want to know the daily ranking of hashtags, so our function is `func.rank()` and we want the hashtags ordered by descending `'count'` column values.

In [ ]:
daily_rank = func.rank() \
                 .over(daily_window.orderBy(func.desc('count'))) \
                 .alias('rank')

So here are the top five hashtags for a few days:

In [ ]:
(day_counts.select('day', 'count', 'hashtag', daily_rank)
           .filter('rank <= 5')
           .show(20))

### A more complicated `Window` example: calculating the daily standard deviation based on a weekly window

The above list of top daily hashtags is interesting, but it really only uncovers very frequent hashtags (e.g. "ff" for "friend forward" or "rt" for "retweet"). We want to try and identify "world events" and for that we want to detect a rapid *change* in the tweeting pattern. 

One very simple model to do this is to just look at the standard deviation of daily tweets. We want to have some sort of reasonably fine-grained control, so we will look at top hashtags per week. Remember standard deviation is just:

$$\sigma = \sqrt{\frac{1}{N}\sum_i{(x_i-\mu)^2}},$$

where $x_i$ is the daily count of a hashtag, $\mu$ is the weekly average and $N$ is the number of samples (here $N=7$ since there are always seven days in a week). 

So the procedure is something like this: 

1. calculate $\mu$ for each hastag and each week
2. compute the sum and take the square root
3. order tweets for each week in order of decreasing $\sigma$

Using standard `RDD`s, the first step would be the most time consuming -- we would need to do an expensive `reduceByKey` on `(week, hashtag, count)` key-value pairs and then either join this result with the original `RDD` or broadcast it as a lookup-table (depending on its size). 

Using a `DataFrame`, we can instead use a "window" function to do the same thing. 

#### Constructing the window function and calculating $\sigma$

Our procedure to calculate $\sigma$ using a weekly mean is as follows:

1. define a `Window` function that will group the data by hashtag and by week
2. compute `mean_diff` which is the difference between the data point and the weekly mean using the window function
2. calculate `var_arg`, which is just $1/7 
3. use `groupBy` to sum up the `sum_arg` column for each hashtag each week
4. calculate `sigma` by taking the square root of the variance 

In [ ]:
# define the window function, partitioned over 'week' and 'hashtag'
weekly_window = Window.partitionBy('week', 'hashtag')

In [ ]:
# define the mean_diff function which computes the difference between each data point 'count' 
# and the mean of 'count' over the weekly window (defined above)
mean_diff = day_counts['count'] - func.mean('count').over(weekly_window)

var_arg = 1./7*func.pow(mean_diff, 2)

In [ ]:
weekly_stats = (day_counts.withColumn('var_arg', var_arg)
                         .groupBy('week', 'hashtag')
                         .sum('var_arg')
                         .withColumnRenamed('sum(var_arg)', 'variance')
                         .withColumn('stddev', func.sqrt('variance'))
                         .sort('stddev', ascending=False))

In [ ]:
weekly_stats.show()

#### Top weekly hashtags by $\sigma$

To get the hashtags that had the most abrupt changes in their usage, we need to rank the hashtags for each week by the standard deviations that we calculated above. Here we use the same pattern as before for getting the top daily tweets:

1. window over the `'week'` and order by '`stddev`'
2. create a column of rank computed over this window
3. display the top 10 for several weeks in a row

In [ ]:
weekly_stats_window = Window.partitionBy('week').orderBy(func.desc('stddev'))

In [ ]:
print('Top ten hashtags with highest variation each week')
weekly_stats.select('week', 'hashtag', 'stddev', 
                     func.rank().over(weekly_stats_window).alias('rank')) \
            .filter('rank < 10') \
            .show(150)

A lot of these are pop-culture events and probably PR campaigns, e.g. 'votefifthharmony' and 'vote5sos'. Turns out the MTV Europe Music Awards were happening during November 2014, so these are not surprising. Also, who knew Twitter was so big in Argentina... check out what [kcaaargentina](http://kidschoiceawardsargentina.mundonick.com/) refers to... (We also see another "world event" -- 'halloween' pop up in week 44. 

Interestingly, in week 48 we finally see something that we are after: ["ferguson"](https://en.wikipedia.org/wiki/Ferguson_unrest). This was a huge event covered world-wide and spanned several months -- it actually started in August, but in November the riots returned when the grand jury decided not to indict the police officer who shot Michael Brown. We'll explore this more a bit further down. 

### Plotting the timeseries of hashtag counts

These standard deviation lists we obtained above are nice, but lets actually look at some data. Below we define a simple function for plotting time series data of hashtags. You can specify a single hashtag or a series of hashtags separated by a comma (','). Note that the time-series is normalized by the standard deviation because often the counts for different hashtags differ by orders of magnitude. 

In [ ]:
minmax = day_counts.select(func.min('day').alias('min'), func.max('day').alias('max')).map(lambda r: (r.min,r.max)).collect()[0]

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

def plot_hashtag_timeseries(hashtags): 
    
    if ',' in hashtags : hashtags = map(lambda x: x.strip(), hashtags.split(','))
        
    if type(hashtags) is not list:
        hashtags = [hashtags]
    max_data = 0
    for hashtag in hashtags:
        day_data = day_counts.filter(day_counts['hashtag'] == hashtag).select('day', 'count').sort('day').toPandas()
        max_data = day_data['count'].max()
        plt.plot(day_data['day'],day_data['count']/day_data['count'].std(), label = hashtag, alpha=.5)
        
    plt.xlabel('day of year')
    plt.legend(fontsize='small', loc='upper left')
    plt.xlim(minmax)
interact(plot_hashtag_timeseries, hashtags='ferguson, fergusondecision');

## Identifying hashtag trends with K-Means clustering

The lists above give us some indication of what was big news on Twitter in the last 2.5 months of 2014. We can now use a clustering approach to identify groups of hashtags that may have been a part of the same viral development. 

Spark includes a K-Means clustering algorithm in MLLib, but it doesn't take a `DataFrame` as input. Instead, we have to pass to it an `RDD` of vectors. 

In [ ]:
from pyspark.mllib.linalg import SparseVector

minday, maxday = day_counts.select(func.min('day').alias('min'), func.max('day').alias('max')).map(lambda r: (r.min,r.max)).collect()[0]

Ndays = maxday-minday+1

For each hashtag, we want an array of daily occurence counts -- the procedure is a bit complex, but the steps go as follows:

1. select `'hashtag'`, `'day'` and `'counts'` from `day_counts`
2. map to an RDD of the form `(hashtag, (day, counts))`
3. use `aggregateByKey` to build a list of `(day, count)` pairs for each hashtag
4. use this list of `(day,count)` pairs to create a `SparseVector`

In [ ]:
def add_tuple_to_vector(vec, tup):
    day, count = tup
    vec[day] += count
    return vec

In [ ]:
day_counts_vec = \
    day_counts.withColumnRenamed('count', 'counts') \
              .map(lambda r: (r.hashtag, (r.day-minday, r.counts))) \
              .aggregateByKey(np.zeros(Ndays), add_tuple_to_vector, lambda a,b: a+b) 

day_counts_vec.cache()

In [ ]:
day_counts_vec.first()

To filter out some of the noise, we'll get rid of everything with a mean count of less than 10 and with 'followme' in the name:

In [ ]:
day_counts_filtered = day_counts_vec.filter(lambda (h,vec): (vec.mean() > 10) & ('followme' not in h)).mapValues(lambda v: v/v.std()).cache()

### Building the model

Now we have our RDD of vectors and we can feed it directly to the [KMeans clustering](https://en.wikipedia.org/wiki/K-means_clustering) algorithm from [Spark's MLlib](http://spark.apache.org/docs/latest/mllib-guide.html).

In [ ]:
from pyspark.mllib.clustering import KMeans
model = KMeans.train(day_counts_filtered.values(), 500, runs=10, maxIterations=1000, initializationMode='random')

After we compute the model, we calculate which cluster each hashtag belongs to:

In [ ]:
clusters = day_counts_filtered.mapValues(lambda vec: model.predict(vec)).cache()

Find the cluster for `'ferguson'` and plot the timeseries for a few hashtags from that cluster:

In [ ]:
ferguson_cluster = clusters.lookup('ferguson')[0]

hashtags = clusters.filter(lambda (hashtag, cluster): cluster == ferguson_cluster).keys().takeSample(False, 5, None)

days = range(minday, maxday+1)
for hashtag in hashtags: 
    vec = day_counts_filtered.lookup(hashtag)[0]
    plt.plot(days, vec, label = hashtag, alpha = .5)
plt.ylabel('normalized frequency'); plt.xlabel('days');
plt.legend(fontsize='small', loc='upper left');

Depending on which hashtags are randomly selected, these may or may not make sense, but for the most part you should see that they all peak at about the same time. 

In [ ]:
def extract_hashtag_cluster(hashtag):
    cluster_id = clusters.lookup(hashtag)[0]
    return clusters.filter(lambda (hashtag, cluster): cluster == ferguson_cluster).keys().collect()

interact(extract_hashtag_cluster, hashtag='ferguson');

These are obviously not all related to Ferguson, but many are (it's nice that it catches the misspelling of "ferguson" also...). You can see hashtags like "fergusondecision", "justiceformikebrown", "ripmikebrown" etc. that are all clearly related. 

Feel free to build your own algorithm and experiment to clean up the clustering results!

### Word2Vec

<div class="alert alert-info" style="margin=10px"><p>Only do the following section if a) you have plenty of time and b) you have a driver with many cores and lots of memory</p></div>

This section is here just for fun because word2vec is implemented in Spark and is fun to play with -- it takes a while to train the model, so only run these next few cells if you're willing to wait a while (~15 minutes on 20 executors). You also need more memory for the driver, so if you followed up to here from the top, you should make sure that your driver has several Gb of RAM. 

[Word2Vec](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) is a popular natural language processing technique that projects words into a vector space that provides information about their context. It can then be used to complete such phrases as, for example, "woman is to queen as man is to X?" The answer obviously being "king" -- in vector space, the separation of "woman" and "queen" is similar to the offset between "man" and "king". 

In [ ]:
hashtag_list_df = (data.select('created_at', 'entities.hashtags')
                .filter(func.size('hashtags') > 0)
                .select(hash_text_udf('hashtags').alias('hashtags'), 'created_at'))

In [ ]:
hashtag_list_df.cache()

In [ ]:
from pyspark.ml.feature import Word2Vec

In [ ]:
npartitions = sc.defaultParallelism*10

In [ ]:
w2v = Word2Vec(numPartitions=npartitions, inputCol='hashtags', outputCol='model')

In [ ]:
%time model = w2v.fit(hashtag_list_df)

In [ ]:
model.findSynonyms('ferguson', 20).show()

In [ ]:
model.findSynonyms('justinbieber', 20).show()

In [ ]:
sc.stop()